# House price prediction

In a project, train a linear regression model on California housing data in 1990. 

Based on the data, you need to predict the median cost of a house in a residential area. 

Train the model and make predictions on the test set. 

Use the RMSE, MAE, and R2 metrics to evaluate the quality of a model.


## Data load and preprocessing

In [1]:
import pandas as pd 
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
    
    
spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

In [2]:
path = 'housing.csv'
df_housing = spark.read.option('header', True).option('inferSchema', True).csv(path)
df_housing.printSchema() 

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [3]:
print(pd.DataFrame(df_housing.dtypes, columns=['column', 'type']))

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string


**Missing valuse**

In [4]:
df_housing.describe().show()

+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|summary|          longitude|         latitude|housing_median_age|       total_rooms|    total_bedrooms|        population|       households|     median_income|median_house_value|ocean_proximity|
+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|  count|              20640|            20640|             20640|             20640|             20433|             20640|            20640|             20640|             20640|          20640|
|   mean|-119.56970445736148| 35.6318614341087|28.639486434108527|2635.7630813953488| 537.8705525375618|1425.4767441860465|499.5396802325581|3.8706710029070246|206855.81690891474|           null|
| stddev|  2.0035317

In [5]:
df2 = df_housing.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_housing.columns])
df2.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



In [6]:
# fill missing values with 0 in total_bedrooms 
df_housing = df_housing \
                .withColumn("total_bedrooms", F.when((F.col("total_bedrooms").isNull()), 0) \
                            .otherwise(F.col("total_bedrooms")))

In [7]:
df_housing.describe().show()

+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|summary|          longitude|         latitude|housing_median_age|       total_rooms|    total_bedrooms|        population|       households|     median_income|median_house_value|ocean_proximity|
+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|  count|              20640|            20640|             20640|             20640|             20640|             20640|            20640|             20640|             20640|          20640|
|   mean|-119.56970445736148| 35.6318614341087|28.639486434108527|2635.7630813953488|   532.47621124031|1425.4767441860465|499.5396802325581|3.8706710029070246|206855.81690891474|           null|
| stddev|  2.0035317

**Transform the ocean_proximity column with categorical values using the One hot encoding technique**

In [8]:
# type
indexer = StringIndexer(inputCol="ocean_proximity", outputCol="ocean_proximity_category")
df_housing_indexed = indexer.fit(df_housing).transform(df_housing)
df_housing_indexed.show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_category|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|                     3.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|                     3.0|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|                     3.0|
+---------+--------+--

In [9]:
encoder = OneHotEncoder(inputCols=["ocean_proximity_category"],
                        outputCols=["ocean_proximity_ohe"])
model = encoder.fit(df_housing_indexed)
df_housing_encoded = model.transform(df_housing_indexed)
df_housing_encoded.show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_category|ocean_proximity_ohe|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------------------+-------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|                     3.0|      (4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|                     3.0|      (4,[3],[1.0])|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177

**Scaling**

In [10]:
numerical_cols  = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]
target = "median_house_value" 

In [11]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
df = numerical_assembler.transform(df_housing_encoded) 

In [12]:
standardScaler = StandardScaler(inputCol='numerical_features', outputCol="numerical_features_scaled")
df = standardScaler.fit(df).transform(df)

In [13]:
print(df.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'ocean_proximity_category', 'ocean_proximity_ohe', 'numerical_features', 'numerical_features_scaled']


In [14]:
all_features = ['ocean_proximity_ohe','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 
df = final_assembler.transform(df)

df.select(all_features).show(3) 

+-------------------+-------------------------+
|ocean_proximity_ohe|numerical_features_scaled|
+-------------------+-------------------------+
|      (4,[3],[1.0])|     [-61.007269596069...|
|      (4,[3],[1.0])|     [-61.002278409814...|
|      (4,[3],[1.0])|     [-61.012260782324...|
+-------------------+-------------------------+
only showing top 3 rows



## Model

In [15]:
RANDOM_SEED = 42

In [16]:
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count()) 

16560 4080


*Using all the data to train the model*

In [17]:
lr = LinearRegression(labelCol=target, featuresCol='features', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(train_data)

23/02/14 08:13:57 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/02/14 08:13:57 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [18]:
trainingSummary = lrModel.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
print("MAE: %f" % trainingSummary.meanAbsoluteError)

RMSE: 68891.490885
r2: 0.639886
MAE: 49894.401039


*Using only numerical data to train the model*

In [19]:
lr_num = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel_num = lr_num.fit(train_data)

In [20]:
trainingSummary_num = lrModel_num.summary
print("RMSE: %f" % trainingSummary_num.rootMeanSquaredError)
print("r2: %f" % trainingSummary_num.r2)
print("MAE: %f" % trainingSummary_num.meanAbsoluteError)

RMSE: 69527.589961
r2: 0.633205
MAE: 50864.183322


**RMSE and MAE metrics are higher for a numeric-only model**

## Test

Check the model with all the data on the test sample

In [21]:
test_result = lrModel.evaluate(test_data)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result.meanAbsoluteError)
print("r2 = %g" % test_result.r2)



Root Mean Squared Error (RMSE) on test data = 71809.2
Mean Absolute Error (MAE) on test data = 51390.4
r2 = 0.627305


Check the model with only numerical data on the test sample

In [22]:
test_result_num = lrModel_num.evaluate(test_data)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result_num.rootMeanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result_num.meanAbsoluteError)
print("r2 = %g" % test_result_num.r2)


Root Mean Squared Error (RMSE) on test data = 72160.4
Mean Absolute Error (MAE) on test data = 52106.5
r2 = 0.623651


### **Resume**

The model trained only on numerical data showed higher values of the main metrics RMSE and MAE